In [1]:
#Standard imports
import numpy as np
import healpy as hp
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.cosmology import FlatLambdaCDM
import fitsio
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.colors import colorConverter
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

from vast.voidfinder import find_voids, filter_galaxies

from vast.voidfinder.multizmask import generate_mask
from vast.voidfinder.preprocessing import file_preprocess

from vast.voidfinder.constants import c

import pickle

from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder.voidfinder_functions import xyz_to_radecz
from vast.voidfinder.voidfinder import ra_dec_to_xyz
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker

from IPython.display import clear_output


In [2]:
import sys
sys.path.insert(1, '/global/homes/h/hrincon/python_tools')
import VoidOverlap as vo

In [3]:
min_z = 0
max_z = 0.24
Omega_M = 0.315
dist_limits = z_to_comoving_dist(np.array([min_z,max_z]).astype(np.float32),Omega_M,1)

In [5]:
masks_path = '/global/homes/h/hrincon/DESIVAST/galaxy_catalog/mask/iron_mask.fits'
mask = fitsio.read(masks_path)
nside=hp.npix2nside(len(mask)) 

def select_hp_mask (gals):
    pxid=hp.ang2pix(nside, gals['RA'], gals['DEC'], nest=True,lonlat=True)
    #Select the galaxies that fall within the mask
    select = np.isin(pxid, mask[mask["DONE"]==1]["HPXPIXEL"])
    return gals[select]

# AltMTL

In [18]:
# Generate mocks
for mock_idx in range (25):
    clear_output(wait=True)
    print('mock #', mock_idx)
    t1 = Table.read(f'/global/cfs/cdirs/desi/survey/catalogs/Y1/mocks/SecondGenMocks/AbacusSummitBGS_v2/altmtl{mock_idx}/mock{mock_idx}/LSScats/BGS_BRIGHT_full_HPmapcut.dat.fits')
    t1=t1[t1['ZWARN']==0]
    # We apply no K or E corrections before taking the mag cut, since the mocks
    # underestimate the Y1 DESIVAST galaxy population, and K+E corrections only make 
    # this problem worse. We're more intested in the differences between mocks than 
    # having them replicate the galaxy population properties accurately, so this is ok
    
    t1=t1[t1['R_MAG_ABS']<-20]
    t1=t1[t1['TRUEZ']<max_z]
    t1=t1[t1['TRUEZ']>min_z]
    t1=select_hp_mask(t1)
    t1=t1['RA','DEC','TRUEZ','R_MAG_ABS']
    t1['RA'].name,t1['DEC'].name,t1['TRUEZ'].name,t1['R_MAG_ABS'].name='ra','dec','redshift','rabsmag'
    
    ngc = (t1['ra'] > 82)*(t1['ra']< 302)
    
    #t1['Rgal']=z_to_comoving_dist(t1['redshift'].astype(np.float32),Omega_M,1)
    mock_name=f'altmtl{mock_idx}'
    t1[ngc].write('./altmtl/'+mock_name+'_ngc.fits')
    t1[~ngc].write('./altmtl/'+mock_name+'_sgc.fits')
    



0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done


In [12]:
#Generate VoidFinder Catalogs
altmtl_path = '/global/homes/h/hrincon/DESIVAST/mocks/altmtl/'

num_cpus = 1
in_directory = altmtl_path
out_directory = altmtl_path
dist_metric = 'comoving'
magnitude_limit = -20.0
min_z, max_z = 0, 0.24001

for idx in range (25):
    for cap in ('ngc','sgc'):
        
        clear_output(wait=True)
        print('mock #', idx)

        mock_name=f'altmtl{idx}_{cap}'
        survey_name = mock_name+'_'
        #galaxy_data_table = Table.read(altmtl_path + mock_name + '.fits')
        galaxy_data_table, dist_limits = file_preprocess(
            mock_name + '.fits', 
            survey_name,
            in_directory, 
            out_directory, 
            #mag_cut=mag_cut,
            dist_metric=dist_metric,
            min_z=min_z, 
            max_z=max_z,
            Omega_M=Omega_M,
            #h=h,
        )

        mask, mask_resolution = generate_mask(galaxy_data_table, 
                                          max_z, 
                                          survey_name,
                                          out_directory,
                                          dist_metric=dist_metric, 
                                          smooth_mask=True,
                                          #h=h,
                                          )

       
        wall_coords_xyz, field_coords_xyz = filter_galaxies(galaxy_data_table,
                                                            survey_name,
                                                            out_directory,
                                                            dist_limits=dist_limits,
                                                            magnitude_limit=magnitude_limit,
                                                            write_table=False,
                                                            dist_metric=dist_metric,
                                                            capitalize_colnames=True,
                                                            #h=h,
                                                            )
        print("Number of filtered galaxies: ", len(wall_coords_xyz)+len(field_coords_xyz))
        del galaxy_data_table
        

        coords_min = np.min(np.concatenate([wall_coords_xyz, field_coords_xyz]), axis=0)


        find_voids(wall_coords_xyz, 
                   survey_name,
                   out_directory,
                   mask_type='ra_dec_z',
                   mask=mask, 
                   mask_resolution=mask_resolution,
                   dist_limits=dist_limits,
                   grid_origin=coords_min,
                   #save_after=50000,
                   #use_start_checkpoint=True,
                   verbose=1,
                   num_cpus=num_cpus,
                   capitalize_colnames=True)

        

mock # 24
Loading galaxy data table at:  /global/homes/h/hrincon/DESIVAST/mocks/altmtl/altmtl24_sgc.fits
Galaxy data table load time:  0.009149551391601562
Calculating Rgal data table column
Finished Rgal calculation time:  0.19103145599365234
Generating mask
Filter Galaxies Start
Checking for duplicate galaxies
Time to check for and remove duplicate galaxies: 0.15548229217529297
Finding isolated galaxy distance
Time to find isolated galaxy distance: 0.19414544105529785
Removing all galaxies with 3rd nearest neighbors further than 9.62486295670853
Number of field gals: 3101 Number of wall gals: 67040
Number of filtered galaxies:  70141
Growing holes
Input galaxies shape:  (67040, 3)
Grid origin: [   7.64458096 -532.43920636 -223.58022528]
Survey-containing box size: [ 662.92102162 1181.30591877  593.09835725]
Hole-growing Grid: (133, 237, 119)
Galaxy-searching Grid: (45, 79, 40)
Galaxy-searching edge length: 15.0
Number of filtered out hole-growing cells: 34839
Total slots in hole_cell

In [ ]:
#Generate V2 catalogs

import configparser, os

from vast.vsquared.zobov import Zobov

altmtl_path = '/global/homes/h/hrincon/DESIVAST/mocks/altmtl/'
config_file = altmtl_path+'ini/vsquared.ini'

for idx in range (25):
    for cap in ('ngc','sgc'):
        clear_output(wait=True)
        
        parser = configparser.SafeConfigParser()
        parser.read(config_file)
        
        parser.set('Paths', 'Input Catalog', altmtl_path + f'altmtl{idx}_{cap}.fits')
        parser.set('Paths', 'Survey Name', f'altmtl{idx}_{cap}_')

        # Writing our configuration file to 'example.ini'
        with open(config_file, 'w') as configfile:
            parser.write(configfile)


        newZobov = Zobov(config_file,
                         start=0,
                         end=3,
                         save_intermediate=False,
                         visualize=True,
                         periodic=False,
                         xyz=False,
                         capitalize_colnames=True)

        #VIDE
        newZobov.sortVoids(method=0)
        newZobov.saveVoids()
        newZobov.saveZones()
        #newZobov.preViz()

        #REVOLVER
        newZobov.sortVoids(method=4)
        newZobov.saveVoids()
        newZobov.saveZones()
        #newZobov.preViz()

Extracting data...
Applying magnitude cut...
Generating mask...
Tesselating...
Computing volumes...
Triangulating...
Consolidating neighbors...
Sorting cells...
Building zones...
Linking zones...


# Horizon Run 4

In [ ]:
#quaternary indexing
idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)
    

In [21]:
# V2 catalogs are computed in parallel in seprate script
# /global/homes/h/hrincon/DESIVAST/mocks/HorizonRun4/voidfinding/vsquared.py

#VoidFinder
hr4_path = '/global/homes/h/hrincon/DESIVAST/mocks/HorizonRun4/'

#quaternary indexing
idx='000'
idx_list = [idx]
for i in range(24):
    idx = idx[0]+idx[1]+str(int(idx[2]) + 1)
    if int(idx[2]) == 4:
        idx = idx[0] +  str(int(idx[1]) + 1) + '0'

        if int(idx[1]) == 4:
            idx = str(int(idx[0]) + 1) + '00'
    idx_list.append(idx)


num_cpus = 1
in_directory = hr4_path
out_directory = hr4_path
dist_metric = 'comoving'
magnitude_limit = None
min_z, max_z = 0, 0.114
mag_cut=False

for idx in idx_list:


    clear_output(wait=True)
    print('mock #', idx)

    mock_name=f'DR7m_{idx}.fits'
    survey_name = f'HR4_{idx}_'
    #galaxy_data_table = Table.read(altmtl_path + mock_name + '.fits')
    galaxy_data_table, dist_limits = file_preprocess(
        mock_name, 
        survey_name,
        in_directory, 
        out_directory, 
        #mag_cut=mag_cut,
        dist_metric=dist_metric,
        min_z=min_z, 
        max_z=max_z,
        Omega_M=Omega_M,
        redshift_name='z'
        #h=h,
    )

    mask, mask_resolution = generate_mask(galaxy_data_table, 
                                      max_z, 
                                      survey_name,
                                      out_directory,
                                      dist_metric=dist_metric, 
                                      smooth_mask=True,
                                      #h=h,
                                      )


    wall_coords_xyz, field_coords_xyz = filter_galaxies(galaxy_data_table,
                                                        survey_name,
                                                        out_directory,
                                                        dist_limits=dist_limits,
                                                        mag_cut=mag_cut,
                                                        magnitude_limit=magnitude_limit,
                                                        write_table=False,
                                                        dist_metric=dist_metric,
                                                        capitalize_colnames=True,
                                                        #h=h,
                                                        )
    print("Number of filtered galaxies: ", len(wall_coords_xyz)+len(field_coords_xyz))
    del galaxy_data_table


    coords_min = np.min(np.concatenate([wall_coords_xyz, field_coords_xyz]), axis=0)


    find_voids(wall_coords_xyz, 
               survey_name,
               out_directory,
               mask_type='ra_dec_z',
               mask=mask, 
               mask_resolution=mask_resolution,
               dist_limits=dist_limits,
               grid_origin=coords_min,
               #save_after=50000,
               #use_start_checkpoint=True,
               verbose=1,
               num_cpus=num_cpus,
               capitalize_colnames=True)



mock # 120
Loading galaxy data table at:  /global/homes/h/hrincon/DESIVAST/mocks/HorizonRun4/DR7m_120.fits
Galaxy data table load time:  0.08664631843566895
Calculating Rgal data table column
Finished Rgal calculation time:  1.7430732250213623
Generating mask
Filter Galaxies Start
Checking for duplicate galaxies
Time to check for and remove duplicate galaxies: 0.6518568992614746
Finding isolated galaxy distance
Time to find isolated galaxy distance: 0.7510585784912109
Removing all galaxies with 3rd nearest neighbors further than 5.635618110583459
Number of field gals: 15536 Number of wall gals: 174009
Number of filtered galaxies:  189545
Growing holes
Input galaxies shape:  (174009, 3)
Grid origin: [-332.1826248  -312.67494271  -32.70992984]
Survey-containing box size: [327.34941171 599.51111496 340.48733149]
Hole-growing Grid: (66, 120, 69)
Galaxy-searching Grid: (22, 40, 23)
Galaxy-searching edge length: 15.0
Number of filtered out hole-growing cells: 60316
Total slots in hole_cell_I